<a href="https://colab.research.google.com/github/lunduniversity/schoolprog-satellite/blob/master/exercises/emission_map/utslappskartan2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Färgkarta med växthusgasutsläpp från Sveriges kommuner

I den här uppgiften ska vi studera hur utsläppen av växthusgaser skiljer sig åt mellan kommunerna i landet. Vi kommer använda data nerladdad från [SMHI](http://www.airviro.smhi.se/cgi-bin/RUS/apub.html_rusreport.cgi), du kan läsa mer om datan på [Länsstyrelsen](http://extra.lansstyrelsen.se/rus/Sv/statistik-och-data/nationell-emissionsdatabas/Pages/default.aspx).

![alt text](https://github.com/lunduniversity/schoolprog-satellite/raw/master/exercises/emission_map/intro_picture.png)

För varje kommun så finns det beräknade värden på hur mycket växthusgaser som släpps ut där. Eftersom det finns flera olika växthusgaser är utsläppen omräknade till hur mycket ren koldioxid det motsvarar. Så kallade *koldioxidekvivalenter*.

Vi kommer att använda den här datan för att rita en karta över Sverige som visar hur mycket växthusgaser det släpps ut per invånare.

Kör kodrutorna i tur och ordning här nedan genom att trycka på pilen till vänster om kodrutan. (Håll musen över kodrutan om du inte ser pilen). Du kan se att du har kört en kodruta genom att det dyker upp ett nummer i stället för pilen. Numren visar i vilken ordning kodrutorna har körts.

Vissa av kodrutorna är stängda. De innehåller kod som du inte behöver förstå. Några rutor är öppna så att man ser koden. Här ska du få ändra senare, men låt dem vara som de är just nu.

In [ ]:
#@title Ladda ner datafiler
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/emission_map/lansrapport_sverige_CO2-equivalents.xls --quiet
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/emission_map/kommuner.geojson --quiet
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/emission_map/befolkning_kommuner_2018.xlsx --quiet

#Utsläpp att kolla på, dessa kommer vi kunna ändra längre fram i uppgiften.
huvudsektor = 'alla'
undersektor = 'alla'

print("Huvudsektor:", huvudsektor)
print("Undersektor:", undersektor)

Kör kodrutan nedan. Om du får massa utskrifter under kodrutan så behöver du  köra kodrutan **ovanför** igen, och sedan kodrutan nedanför en gång till. 

In [ ]:
#@title Läs in data
#här måste vi fixa
import pandas as pd

#Får ett filnamn till utsläpp för sveriges kommuner. Returnerar Dataframe från xls-filen.
def read_data(filename, huvudsektor = 'alla', undersektor = 'alla'):
  df = pd.read_excel(filename, sheet_name=1)
  values = df.values.tolist() #Skapar en matris med innehållet från andra
                              #databladet i excel-filen
  #Väljer ut rader som innehåller data för rätt kategori och underkategori för alla kommuner.
  tabell = [row for row in values if str(row[0])[:3].lower() == huvudsektor[:3].lower() and  row[1] == undersektor and row[3] != "alla" and row[2] != "Rest"]
  header = values[3]
  header[3] = "Kommun"
  kort_tabell = [rad[3:] for rad in tabell]
  return pd.DataFrame(kort_tabell, columns = header[3:])

try: utslapp = read_data("lansrapport_sverige_CO2-equivalents.xls", huvudsektor, undersektor)
except (ImportError):
  print("Installerar programvara. Kör om första kodrutan igen!")
  !pip3 install --upgrade xlrd
  exit()  


befolkning = pd.read_excel("befolkning_kommuner_2018.xlsx", skiprows=[0,1,2,3,5,6,7,8], nrows =  311, header = 0)
befolkning = befolkning.mask(befolkning.Kommun < 100).dropna() #Ta bort län som har låga 'Kommunkoder'
befolkning["Kommun"] = befolkning["Kommun"].astype(int)
befolkning_dict = {}
for index, row in befolkning.iterrows():
  befolkning_dict[row[1].strip()] = row[2]

for index, row in utslapp.iterrows():
  kommun = row.Kommun
  if kommun not in befolkning_dict:
    kommun=kommun.replace("Upplands-Väsby", "Upplands Väsby")
    kommun=kommun.replace("ÖstraGöinge", "Östra Göinge")
    kommun=kommun.replace("LillaEdet", "Lilla Edet")
    kommun=kommun.replace("Malung", "Malung-Sälen")
    row.Kommun = kommun
  for i in range(1,len(row)): # Omvandla absoluta tal till per capita
    row[i]/=float(befolkning_dict[kommun])
  utslapp.iloc[index] = row

#Gör så att man kan indexera med kommunnamn
utslapp = utslapp.set_index('Kommun')

kommungeometri = "kommuner.geojson"

print("Klart!")

In [ ]:
import branca.colormap as cm
fargskala = cm.LinearColormap(
    #Vilka färger som ska vara i färgskalan:
    colors = ['green', 'yellow', 'orange', 'red'],
    #Var dessa färger ska hamna på färgskalan i tur och ordning:
    index = [0, 2, 10, 100],
    #Övre och undre gräns för färgskalan
    vmin = 0,
    vmax = 100,
    caption = "ton CO2 ekvivalent per person",
)

display(fargskala)


Nu borde du ha fått upp en färgskala. Med hjälp av denna ska vi färglägga Sveriges kommuner efter hur mycket växthusgaser de släpper ut. Färgerna är inte jämnt utspridda. Grönt, gult och orange är ganska nära varandra, medan rött är längre bort. Detta är för att vissa kommuner släpper ut väldigt mycket, per person. Med den här skalan kan man dels se skillnad på kommunerna som släpper ut lite grann, och även de kommuner som släpper ut mer.

Vi har satt färgen gul vid 2 ton CO2 per person och år. För att ha en hållbar utsläppsnivå ska man släppa ut ungefär 1-2 ton CO2 per person och år, enligt [Sveriges Natur](https://www.sverigesnatur.org/aktuellt/sverige-inget-klimatforedome-nar-konsumtionen-raknas/). Man kan alltså se det som att kommuner som är mörkgula, orangea eller röda släpper ut mer än vad som är hållbart.

Kör de nästa två kodcellerna så ska du få upp en karta över Sverige.

In [ ]:
#Vilket årtal ska vi använda data från?
#Välj mellan 1990, 2000, 2005 samt åren mellan 2010 och 2017
artal = 2017

print("Årtal:", artal)

In [ ]:
#@title Rita karta
import folium
def style_function(feature):
    kommun = feature['properties']['KOMMUNKOD']

    emission = utslapp.loc[kommun][str(artal)]
    return {
        'fillColor': '#gray' if emission is None else fargskala(emission),
        'fillOpacity': 0.6,
        'weight': 0.2,
    }

world_map = folium.Map(
        location=[63, 15],
        zoom_start=4,
        tiles='openstreetmap'
        )

folium.GeoJson(
    data = kommungeometri,
    style_function = style_function    
).add_to(world_map)
#display(linear)
world_map.add_child(fargskala)
print("Huvudsektor:", huvudsektor)
print("Undersektor:", undersektor)
print("Utsläpp per person för Sveriges kommuner år:", artal)
world_map


Nu borde du ha fått upp en karta. Hur ser det ut? Ligger Sverige bra till eller behöver vi bli bättre?

Ett par kommuner är väldigt röda, och släpper ut i närheten av 100 ton CO2 per person och år. Detta är inte för att de som bor där lever väldigt ohållbart, utan för att det är små samhällen med stora industrier som släpper ut mycket. Zooma in på kartan så att du ser vad orterna heter. Kan du ta reda på vad det är för industrier där som kan tänkas släppa ut mycket växthusgaser? Du kan använda google, wikipedia etc.

<details>
<summary markdown="span">
Svar
</summary>
<p>
I Lysekil, på västkusten, finns Skandinaviens största oljeraffinaderi. I Oxelösund, på östkusten, finns ett järnverk.
</p>
</details>

## Byt årtal

Ändra värdet på variabeln `artal` ovanför till något annat år. År som fungerar är 1990, 2000, 2005 samt åren mellan 2010 och 2017. När du ändrat vilket år som ska visas måste du först köra kodcellen som väljer årtal, och sen den som ritar kartan.

Kan du se någon förändring över tid? Släpper vi ut mer eller mindre här i Sverige än vi gjorde 1990? 

<details>
<summary markdown="span">
Svar
</summary>
<p>
Utsläppen har minskat!<br><br>
Det är ju positivt, men räcker minskningen? Och hur är det med de utsläpp som orsakas av att vi importerar varor och reser utomlands? Läs gärna artikeln i Sveriges Natur som länkades ovan.
</p>
</details>

## Ändra färgskalan

Nu ska du få ändra på färgskalan. Gå upp till kodcellen för färgskalan och prova att ändra i `colors` och `index`. Exempel på vad du kan göra är:
1.   Byta ut en färg mot någon annan, de flesta färger på engelska fungerar.
2.   Flytta var en färg är på skalan genom att ändra i `index`
3.   Byta plats på två färger i `colors`
4.   Lägga till en ny färg. Då måste du lägga till en färg i `colors` och ett tal i `index`.

När du ändrat i kodcellen för färgskalan behöver du köra den och sen rita kartan, så byts färgerna på kartan ut.

## Byt sektor att visa

Som en sista övning kan du ändra på vilken data som visas på kartan. Det gör du i kodcellen där `huvudsektor` och `undersektor` tilldelas. (Nästan längst upp på sidan.)

Du kan börja med att ändra på vilken huvudsektor du vill titta på. Ändra variabeln `huvudsektor` till någon av följande (byt alltså ut texten `'alla'` i koden):

*   `El och fjärrvärme`
*   `Egen uppvärmning av bostäder och
lokaler`
*   `Industri (energi och processer)`
*   `Transporter`
*   `Arbetsmaskiner`
*   `Produktanvändning (inkl.
lösningsmedel)`
*   `Jordbruk`
*   `Avfall (inkl. avlopp)`
*   `Utrikes transporter`

När du har ändrat behöver du köra kodcellen och alla stängda kodceller under för att få upp en ny karta. 

Vissa sektorer står för betydligt mindre utsläpp än totalen. Då kan det vara värt att ändra i färgskalan så att man ser tydligare. Minska `vmax` och `index` så att färgskalan passar sektorn. 

Kan du ta reda på vilken sektor som står för utsläppen i de två kommunerna som var väldigt röda på kartan för hela Sverige?

<details>
<summary markdown="span">
Svar
</summary>
<p>
Om man visar sektorn <code>Industri (energi och processer)</code> så är de två kommunerna fortfarande väldigt röda. För andra sektorer sticker de inte ut. Detta stämmer alltså med vår teori att det är de stora industrierna här som är anledningen till att mycket släpps ut.
</p>
</details>




Du kan också välja undersektor. Några av undersektorerna till huvudsektorn `Transporter` är:

*   `Personbilar`
*   `Tunga lastbilar`
*   `Bussar`
*   `Järnväg`
*   `Militär transport`

Ändra `huvudsektor` till `Transporter` och välj någon av undersektorerna att sätta `undersektor` till. Studera hur mycket/lite som släpps ut av de olika transportmedeln. Längst ner i uppgiften finns alla underkategorier.

## Kommentar

Siffrorna vi visualiserar här säger inte allt. Vi i Sverige importerar mycket från utlandet. Då släpps växthusgaser ut på andra platser än i Sverige, även om det i grund och botten är vi som orsakar utsläppen. Du kan läsa mer om utsläpp i utlandet här: [Sveriges Natur](https://www.sverigesnatur.org/aktuellt/sverige-inget-klimatforedome-nar-konsumtionen-raknas/) (Länken fanns längre upp också) 

# Nu är det dags för ett quiz! 

Du kan använda kartan och koden ovanför för att få svar på frågorna.

In [ ]:
#@title Kör rutan för att öppna quizet!
!wget https://raw.githubusercontent.com/lunduniversity/schoolprog-satellite/master/for_developers/quiz.py --quiet
!wget https://raw.githubusercontent.com/lunduniversity/schoolprog-satellite/master/exercises/emission_map/quiz_emission_map.json --quiet
import json
import quiz

with open("quiz_emission_map.json") as f:
  quiz_dict = json.load(f)
quiz.main(quiz_dict)

# Alla underkategorier

Här hittar du alla underkategorier, grupperade efter huvudkategori. Om huvudkategorin inte är med i listan innebär det att det inte finns underkategorier för den huvudkategorin.



## Egen uppvärmning av bostäder och lokaler


*   Kommersiella och offentliga lokaler
*   Bostäder
*   Jordbruks- och skogsbrukslokaler



## Transporter

*   Personbilar
*   Lätta lastbilar
*   Tunga lastbilar
*   Bussar
*   Mopeder och motorcyklar
*   Inrikes civil sjöfart (inkl. privata fritidsbåtar)
*   Inrikes flygtrafik
*   Järnväg
*   Militär transport

  

## Arbetsmaskiner

*   Industri- och byggsektorns arbetsmaskiner (inkl. vägarbeten)
*   Fiskebåtar
*   Jordbruk och skogsbruk
*   Kommersiella och offentliga verksamheter
*   Övrigt (flygplatser, hamnar, m.m.)
*   Hushållens arbetsmaskiner
*   Skotrar och fyrhjulingar

## Produktanvändning (inkl. lösningsmedel)

*   Färg - hushåll
*   Färg - verksamheter
*   Lösningsmedelsanvändning - verksamheter
*   Smörjmedel
*   Paraffinvax
*   Urea för katalysatorer
*   Lustgas fran produktanvändning
*   Användning av fluorerade gaser


## Jordbruk

*   Djurs matsmältning
*   Kogödsel (lagring, användning och bete)
*   Svingödsel (lagring, användning och bete)
*   Hästgödsel (lagring, användning och bete)
*   Hönsgödsel (lagring, användning och bete)
*   Fårgödsel m.m. (lagring, användning och bete)
*   Användning av konstgödsel
*   Skörderester som gödsel
*   Kalkning av åkermark
*   Odling av organogena jordar
*   Odling av mineraljordar
*   Indirekta utsläpp av lustgas från brukad mark
*   Övriga gödselmedel m.m.

## Avfall (inkl. avlopp)

*   Avfallsdeponier
*   Biologisk behandling av avfall
*   Behandling av avloppsvatten
*   Förbränning av farligt avfall
